# Nixie Voltage Regulator
## Basic Design Parameters
In this section we will define the basic parameters of the Voltage Regulator. We will specify known quantities at this stage such as input voltage, required output current, total power, basic efficiency calculations etc.

In [96]:
from pint import UnitRegistry
ureg = UnitRegistry()

#Input Voltage Range
VinMin = 4.0 * ureg.V
VinNom = 5.0 * ureg.V
VinMax = 6.0 * ureg.V

Vout = 180.0 * ureg.V

#Load Current
NixieCurrent = 2.5 * ureg.mA
NumNixie = 6
# Total current draw of Nixies + some fudge factor.
MaxDesignCurrent = NixieCurrent * NumNixie + 2.5 * ureg.mA

OutputPower = (MaxDesignCurrent * Vout).to("W")
InputCurrent = OutputPower / VinNom

Eff = 0.7

print("Basic Design Parameters".center(80,'='))
print("Assuming efficiency of ",Eff,"%\n")
print("Input Params\tOutput Params")
print(VinNom,Vout,sep='\t')
print(OutputPower/Eff,OutputPower,sep='\t')
print(((OutputPower/Eff)/VinNom).to("A"))



============================Basic Design Parameters=============================
Assuming efficiency of  0.7 %

Input Params	Output Params
5.0 volt	180.0 volt
4.5 watt	3.15 watt
0.9 ampere


We can see that the power requirements for the converter are quite low at less than 5 watt. This means that the converter should be capable of being powered from a suitably powerful USB source such as a mobile phone charger or power bank.

### Boost Converter in CCM Mode
Firstly, we'll calculate what the solution will look like in CCM mode as it's simpler to work out and our load is relativley constant and well understood.

### Duty Cycle

Duty Cycle in CCM is defined as the following.

$D=1-\dfrac{V_{in}}{V_{out}+V_d}$

where $V_d$ is the forward voltage of the boost diode.

In [97]:
#Diode Voltage Drop
VDropD = 1 * ureg.V # Assuming 1V for now.

Duty = 1 - ((VinMin)/(Vout+VDropD))

print('{:.4f}'.format(Duty.magnitude))

0.9779


A 97% Duty Cycle is really high, but do-able for some converters. We'll be spending most of our time with the MOSFET switched on so it'll need to have a good RDSon.

### Inductor Selection

For a Boost Converter to operate in CCM there must be a minumum load at the output in order to maintain a constant current through the inductor to prevent the converter from switching into DCM. Because the converter is only going to be powering 6 Nixie tubes and will be shut down when I don't want the tubes to be lit then we can rely on a fairly constant load.

However, if PWM brightness control is implemented (which it will) then CCM will not be suitable and a DCM mode Boost of Flyback will be needed instead.

For now lets push on and pretend this isnt a factor.